<span style="font-size:13px;">

##### ▶ 텍스트 벡터화 : 텍스트 데이터를 메신러닝 모델이 이해할 수 있는 숫자 형태의 벡터로 변환하는 기법
-  BOW (Bag of Words) CountVectorizer
    - 문서를 고정된 길이의 벡터로 변환
    - 문서 - 단어행렬
    - 장점 : 간단하고 빠름,
    - 단점 : 단어순서 손실, 희소성, 의미적 유사성 무시

-  TF-IDF TfidVetorizer
    - 모든 문서에서 자주 등장하는 단어의 영향을 줄이고, 문서의 특이 단어를 강조

-  N-gram 
    - 단어의 순서를 보존하기 위해 N개의 연속된 단어 시퀀스를 하나의 단위로 보는 것
    - 단점 차원폭발에 주의 (정규화/ 차원 축소 고려)

##### ▶ 머신러닝 분류 모델 : 벡터화된 텍스트 데이터를 기반으로 문서를 특정 카테고리로 분류하는 알고리즘
-  multinomal Naive Bayes (다항분포 나이브 베이즈)
    - 확률 모델

-  LogisticRegression 
    - 회귀를 기반으로하지만 시그모이드 함수를 사용하여 클래스를 분류하는데 널리 사용되는 모델
    - 다중클래스 회귀기반 분류

- RidgeClassifer
    - L2규제를 적용한 회귀 기반 분류 모델로 모델의 과적합을 방지


##### ▶ 한국어 형태소 분석기 : 한국어 텍스트를 처리하기 위한 라이브러리
- Kolnpy Okt
---

### <span style="color: gold;">1. 데이터 준비

In [1]:

import numpy as np
import pandas as pd
# scikit-learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
#분류모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:

categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# # data load
# newsgroups_train =  fetch_20newsgroups(subset='train'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )
# newsgroups_test =  fetch_20newsgroups(subset='test'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )

In [4]:
from sklearn.datasets import load_files
train_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-train'
test_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-test'
newsgroups_train = load_files(train_path,encoding='latin1')
newsgroups_test = load_files(test_path,encoding='latin1')

In [5]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories

In [6]:
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_test,categories)

In [9]:
np.unique(train_target)

array([0, 1, 2, 3])

---
### <span style="color: gold;"> 2. 텍스트 전처리

In [7]:
# 헤더 푸터 인용문 제거
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)


In [8]:
# 텍스트 전처리
train_data = [ clean_text(t) for t in train_data]
test_data = [ clean_text(t) for t in test_data]

In [10]:
len(train_data), len(train_target), len(test_data), len(test_target)

(970, 970, 0, 0)

<span style="font-size:13px;">

#### 멀티노멀 나이즈베이즈  
    - 단어의 **출현 횟수(빈도)**를 세어서, 주어진 문서가 특정 카테고리(예: 스팸, 정상)에 속할 확률을 계산하는 방법
    - 활용분야: 스팸필터링, 뉴스기사 카테고리, 감성분석  
#### 1.  베이즈정리 : 확률 이론 -> 조건부 확률
- 단어 A가 나왔을 때 이 문서가 스팸 B일 확률은 얼마인가  
- $P(\text{스팸} | \text{단어들}) = \frac{P(\text{단어들} | \text{스팸}) \cdot P(\text{스팸})}{P(\text{단어들})}$

#### 2. 나이브 Naive
- 가정: 문서 안의 모든 단어는 서로 독립적이라고 가정
- 현실: 스팸에 자주 나오는 단어들은 서로 독립적이지 않다
- 실제: 이러한 가정은 계산량을 빠르게 하고 단순하지만 정확도가 어느정도 나온다

#### 3. 멀티노멀 : 다항분포
- 의미 : 
    - '다항 분포'를 의미하며, 단어의 출현 횟수를 가장 중요한 정보로 사용
    -  어떤 단어가 단순히 '있다/없다'가 아니라 '몇 번 나왔는가'
- 회수를 세는 멀티노미얼 방식이 NLP와 잘 맞는다
- 모델은 단어의 빈도수 통계
---
- 예시 - 이러한 통계를 바탕으로 이 카테고리에서 특정 단어가 나올 확률 P('free'|스팸)을 모두 계산
-   **스팸 메일 통계**
    -   `free`: 150번
    -   `money`: 100번
    -   `viagra`: 50번
    -   `report`: 5번
-   **정상(Ham) 메일 통계**
    -   `report`: 80번
    -   `meeting`: 60번
    -   `free`: 10번
-   **새로운 메일 분류**  'free money metting'

**1. 이 메일이 '스팸'일 확률 점수 계산**

> **점수(스팸) = P(스팸) × P('free'|스팸) × P('money'|스팸) × P('meeting'|스팸)**  
        -> 기본스팸확률 x 스팸일때 free가 나올 확률 x 스팸일때 money가 나올 확률 x 스팸일 때 meeting 나올 확률                   

-   `P(스팸)`: 전체 메일 중 스팸 메일이 차지하는 기본 확률
-   `P('free'|스팸)`: 스팸 메일에서 'free'가 나올 확률
-   `P('money'|스팸)`: 스팸 메일에서 'money'가 나올 확률
-   `P('meeting'|스팸)`: 스팸 메일에서 'meeting'이 나올 확률

**2. 이 메일이 '정상'일 확률 점수 계산**

> **점수(정상) = P(정상) × P('free'|정상) × P('money'|정상) × P('meeting'|정상)**  
        -> 기본 정상확률 X 정상일때
-   `P(정상)`: 전체 메일 중 정상 메일이 차지하는 기본 확률
-   `P('free'|정상)`: 정상 메일에서 'free'가 나올 확률
-   `P('money'|정상)`: 정상 메일에서 'money'가 나올 확률
-   `P('meeting'|정상)`: 정상 메일에서 'meeting'이 나올 확률

---
### <span style="color: gold;"> 3. 텍스트 벡터화

In [48]:
# nltk tokenizer stemer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAMSUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:

# min_df : 단어의 빈도가 최소 5개의 문서에 등장  - 노이즈 감소
# max_df : 50% 너무흔한 단어는 제거
cv = CountVectorizer(max_features=2000,min_df=5, max_df=0.5)
x_train_cv = cv.fit_transform(train_data)
x_test_cv = cv.transform(test_data)
x_train_cv.shape,  x_test_cv.shape

AttributeError: 'NoneType' object has no attribute 'lower'

---
### <span style="color: gold;"> 4. 모델 학습 및 평가

<span style="color: pink">1. MultinomialNB (다항분포 나이브 베이즈)</span>

- BoW(CountVectorizer) + MultinomialNB    
- TF-IDF + MultinomialNB    


In [119]:
# BOW 기반 + 멀티노미얼 나이브베이즈 MNB
# 텍스트 분류의 강력한 BASELINE 희소데이터에 강함
# 모델선택
nb = MultinomialNB()
# 학습용데이터 벡터데이터
nb.fit(x_train_cv,train_target)
print(nb.score(x_train_cv, train_target)  , nb.score(x_test_cv,test_target))
# 분류 리포트
from sklearn.metrics import classification_report
y_pred_nb = nb.predict(x_test_cv)
print( classification_report(test_target,y_pred_nb,target_names=categories)  )

0.9508357915437562 0.861049519586105
                    precision    recall  f1-score   support

       alt.atheism       0.80      0.79      0.80       319
talk.religion.misc       0.74      0.73      0.73       251
     comp.graphics       0.92      0.95      0.93       389
         sci.space       0.93      0.91      0.92       394

          accuracy                           0.86      1353
         macro avg       0.85      0.85      0.85      1353
      weighted avg       0.86      0.86      0.86      1353



In [122]:
# TF-IDF + 멜티노비얼 나이브베이즈MNB + 노지스틱레그레이션 LogiscticRegression
# TF-IDF로 중요단어 강조, 선형모델과 자주사용 BOW 대비 흔한 단어 영향 감소
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_tfid = tfidf.fit_transform(train_data)
x_test_tfid = tfidf.transform(test_data)

# NB + tf-idf 
nb_tfidf  = MultinomialNB()
nb_tfidf.fit(x_train_tfid,train_target)
print(nb_tfidf.score(x_train_tfid,train_target),  nb_tfidf.score(x_test_tfid, test_target) )

0.9513274336283186 0.8566149297856614


<span style="color: pink">2. LogisticRegression (로지스틱 회귀)</span>
-  TF-IDF + LogisticRegression (기본, L2 규제)
-  TF-IDF + LogisticRegression (L1 규제)

In [123]:
# logistic regression + tf-idf
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train_tfidf, train_target, test_size=0.2,stratify=train_target, random_state=42)
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)
print(lr.score(x_train,y_train), lr.score(x_test,y_test))


0.9834050399508297 0.9434889434889435


<span style="color: pink">3. RidgeClassifier (릿지 분류기)</span>

In [71]:
# 과적합 해결을 위한 규제
rc = RidgeClassifier(alpha=5)
rc.fit(x_train_tfidf, train_target)
print(rc.score(x_train_tfidf,train_target), rc.score(x_test_tfidf,test_target)) 

0.9759095378564405 0.8625277161862528


In [76]:
# L1 규제L1 Logistic (Lasso와 유사)
# 일부 계수를 0으로 만들어서 특성 선택을 수행 ... 주요 피처 선택 효과
l1_lr = LogisticRegression(penalty= 'l1', max_iter=1000, solver='liblinear')
l1_lr.fit(x_train, y_train)
print(l1_lr.score(x_train,y_train), l1_lr.score(x_test,y_test))

0.9373079287031346 0.914004914004914


c:\Users\SAMSUNG\anaconda3\envs\deep\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


<span style="color: pink">3. 트리 기반 앙상블 모델</span>
- DecisionTreeClassifier( 결정트리)
- RandomForestClassifier (랜덤 포레스트)
- GradientBoostingClassifier (그래디언트 부스팅)

In [75]:
# 트리모델 + TF-IDF
tree = DecisionTreeClassifier(max_depth=3)
fores = RandomForestClassifier(max_depth=3)
gb = GradientBoostingClassifier(max_depth=3)

tree.fit(x_train, y_train)
fores.fit(x_train, y_train)
gb.fit(x_train, y_train)

print(tree.score(x_train,y_train), tree.score(x_test,y_test))
print(fores.score(x_train,y_train), fores.score(x_test,y_test))
print(gb.score(x_train,y_train), gb.score(x_test,y_test))


0.5138291333743086 0.4864864864864865
0.7572218807621389 0.7272727272727273
0.9987707437000615 0.9287469287469288


In [82]:
x_train,x_test,y_train,y_test = train_test_split(train_data, train_target,stratify=train_target, test_size=0.2, random_state=42)

In [125]:

# 전처리 
# RegexpTokenizer + stopwords + PorterStemmer
english_stops = set(stopwords.words('english'))
regtok = RegexpTokenizer(r"[\w']{3,}")
def custom_tokenizer(text):
    toks = regtok.tokenize(text.lower())
    toks = [t for t in toks if t not in english_stops]
    toks = [PorterStemmer().stem(t) for t in toks]
    return toks
tfidf_custom = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000,min_df=5, max_df=0.5)
x_train_tfidf_c = tfidf_custom.fit_transform(x_train)
x_test_tfidf_c = tfidf_custom.transform(x_test)

c:\Users\SAMSUNG\anaconda3\envs\deep\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


AttributeError: 'csr_matrix' object has no attribute 'lower'

In [124]:
from sklearn.linear_model import LogisticRegression
lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_tfidf_c,y_train)
print( lr_c.score(x_train_tfidf_c, y_train) , lr_c.score(x_test_tfidf_c, y_test))

ValueError: Found input variables with inconsistent numbers of samples: [2034, 1627]

In [106]:
# n_gram 실험 1,2 1,3
# 성능향상 기대 연속된 단어패턴 포착
tfidf_12 = TfidfVectorizer(token_pattern= r"[\w'] {3,}", stop_words=stopwords.words('english'),
                           ngram_range = (1,2), min_df=5, max_df=0.5) #max_features=2000)

x_train_12 = tfidf_12.fit_transform(x_train)
x_test_12 = tfidf_12.transform(x_test)
x_train_12.shape#, x_test_12.shape

lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_12, y_train)
print(lr_c.score(x_train_12,y_train), lr_c.score(x_train_12,y_test))

ValueError: Found input variables with inconsistent numbers of samples: [407, 1627]

---

In [ ]:
# 한국어처리 konply
# 품사기반 태깅 tokenizer noun verb adjectiv 

# 데이터로딩
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [114]:
df.title.unique()

array(['인피니티 워', '라라랜드', '곤지암', '신과함께', '범죄도시', '택시운전사', '코코'],
      dtype=object)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.review, df.title, stratify=df.title, test_size=0.2, random_state=42)
from konlpy.tag import Okt
okt = Okt()

[12864    광주민주항쟁은  개인에  영웅담이  아니라 우리민족이 가진  비극적인 역사중 하나이...
 5766     왜 그렇게 인기가 많은지 이해가안됍니다. 중간에 중간 지루함도있고 영화에서 전달하려...
 4858             평점 높은사람들이 원작보면, 다 평점깎을거라 생각한다, 모든걸 망친 각색.
 12728                                             맘이 아픕니다.
 8699     이렇게 한국 영화를 재밌게 보는 건 처음 입니다.스토리도 탄탄하고 한국에서 이런 영...
                                ...                        
 2117                                           쓰레기   시간아깝다
 2810                                                 넘 지루함
 6758     CG는 많이 늘었네 느낌 눈물은 원래 많은 편이라 그냥 울었음 내용은 산만하고 조금...
 10486                                                  bbb
 2954                                       이런 천재감독을 봤나~~대박
 Name: review, Length: 11780, dtype: object,
 4651                      킬링타임용 여러가지 시도를 한것같긴한데 뭔가가 빠진듯...
 9094     원작내용을 잘녹였고 cg도 최선을다한느낌인데 지루함. 걍 지루해. 노잼이 아닐수도있...
 273                                           이 영화는 무조건 백점
 11897    광주시민이 아닌 서울시민의 광주항쟁 경험사. 좀 더 리얼하게 그렸으면 하는 아쉬움....
 4489     미쳣다 이건.. 한국에서 만든게 맞나 싶을 정도로 놀랍다... 너무 무섭고 감탄하

In [117]:
# simple 버전
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:

clf.fit(x_train_tfidf, y_train)
clf.score(x_train_tfidf, y_train), clf.score(x_test_tfidf, y_test)

In [118]:
def custom_tokenizer(text):
    target = ['None','Verb','Adjective']
    return [w for w, tag in okt.pos(text,norm=True, stem=True) if tag in target]

tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)
clf.fit(x_train_tfidf, y_train)
clf.score(x_train_tfidf, y_train), clf.score(x_test_tfidf, y_test)


c:\Users\SAMSUNG\anaconda3\envs\deep\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: empty vocabulary; perhaps the documents only contain stop words